# Link Prediction in Condmat

In [3]:
import collections
import copy
import datetime
import itertools
import math
from typing import List, Any, Dict, Tuple

import joblib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import precision_recall_curve, average_precision_score, roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold, cross_validate, GridSearchCV, train_test_split
import seaborn as sns
from tqdm import tqdm
from xgboost import XGBClassifier

# Typing
NodePair = Tuple[int, int]
Edge = List[Tuple[int, int, Dict['date', datetime.datetime]]]

In [4]:
def read_edges(file: str, sep=' ') -> pd.DataFrame:
  d = pd.read_csv(file, sep, skiprows=1, names=['source', 'target', 'weight', 'date'])
  d['date'] = d['date'].apply(datetime.datetime.fromtimestamp)
  d.sort_values(by='date', inplace=True)
  return d.loc[:, ['source', 'target', 'date']]
def filter_edgelist(edges: pd.DataFrame, start=0, stop=1, verbose=True) -> pd.DataFrame: 
  """Filter edgelist.  If start/ stop is float, start/stop from the fraction of total edges. If datetime, this is used.""" 
  no_edges = len(edges)
  if start != 0:
    if type(start) is float:
      assert 0 < start < 1
      start = int(start*no_edges)
    if type(start) is int: start = edges.iloc[start]['date']
    start = start + datetime.timedelta(seconds=1)
  else: start = edges['date'].min()
  if verbose: print(start)
  
  if stop != 1:
    if type(stop) is float:
      assert 0 < stop < 1
      stop = math.floor(stop*no_edges)-1
    if type(stop) is int: stop = edges.iloc[stop]['date']
  else: stop = edges['date'].max()
  if verbose: print(stop)
  
  mask = (edges['date'] >= start) & (edges['date'] <= stop)
  if verbose: 
    no_selected_edges = sum(mask)
    print(f'{no_selected_edges=}, ({no_selected_edges/len(edges):.1e})')

  return edges.loc[mask]
def convert_to_set(edges: pd.DataFrame) -> List[NodePair]: return {edge for edge in edges.loc[:, ['source', 'target']].itertuples(index=False, name=None)}
def get_graph(edgelist: pd.DataFrame) -> nx.Graph:
  """Add edge to graph. Contains edge attribute weight."""
  g = nx.Graph()
  
  for u, v, _ in edgelist.itertuples(index=False, name=None):
    weight = g[u][v]["weight"]+1 if g.has_edge(u,v) else 1
    g.add_edge(u, v, weight=weight)
  
  return g
def giant_component(graph: nx.Graph) -> nx.Graph: return graph.subgraph(max(nx.connected_components(graph), key=len)).copy()
def report(graph:nx.Graph, probes: Tuple[int, int]):
  n = len(probes)
  print(f"Number of probes: {n}")
  a = sum([graph.has_edge(u, v) for u, v in probes])
  print(f"- already edge: {a} ({a/n:.0%})")
  non_edges = set(nx.non_edges(graph))
  ne = sum([np in non_edges for np in probes])
  print(f"- both nodes in graph: {ne} ({ne/n:.0%})")
  ng = sum([not (graph.has_node(u) and graph.has_node(v)) for u, v in probes])
  print(f"- not in graph: {ng} ({ng/n:.0%})")
def get_distances(graph: nx.Graph, cutoff: int = None) -> (List[NodePair], List[int]):
  """
  Get all non-edges using BFS. When cutoff provided, consider only node pairs with at most this distance.
  Returns:
  - nodepairs: tuple containing all nodepairs
  - distances: tuple containing all distances
  """
  return zip(
    *[
      ((u, v), distance)
      for u, (nbs_u, _) in tqdm(nx.all_pairs_dijkstra(graph, cutoff, weight=None), total=len(graph), desc="get_distances")
      for v, distance in nbs_u.items() if distance > 1 and (cutoff is None or distance <= cutoff) 
    ]
  )

In [6]:
edges = read_edges('out.topology')

In [8]:
print(f'Number of {len(edges):.1e}')

Number of 1.7e+05


In [9]:
edges_mature = filter_edgelist(edges, stop=50000)

2010-02-15 01:00:00
2010-02-15 01:00:00
no_selected_edges=67010, (3.9e-01)


In [10]:
edges_probe = filter_edgelist(edges, start=50000, stop=70000)

2010-02-15 01:00:01
2010-02-15 01:07:33
no_selected_edges=2992, (1.7e-02)


## Set-up
Choose here the parameters on how you want to define the learn and assessing phase.

In [8]:
g_learn = giant_component(get_graph(edges_mature))
uv_assessing = convert_to_set(edges_probe)

In [10]:
joblib.dump(g_learn, 'random/graph.pkl', protocol=5)
joblib.dump(uv_assessing, f'random/probes.pkl', protocol=5)

['random/probes.pkl']

In [11]:
report(graph=g_learn, probes=uv_assessing)

Number of probes: 19805
- already edge: 117 (1%)
- both nodes in graph: 5967 (30%)
- not in graph: 8318 (42%)


## Export

In [ ]:
%%time
targets = joblib.load(f'{folder}all/targets.pkl')
nodepairs = joblib.load(f'{folder}all/nodepairs.pkl')
distances = joblib.load(f'{folder}all/distances.pkl')
g_learn = joblib.load(f'{folder}all/graph.pkl')

In [12]:
nodepairs, _ = get_distances(g_learn, cutoff=2)

get_distances: 100%|██████████| 20981/20981 [00:13<00:00, 1582.25it/s]


In [13]:
targets = [nodepair in uv_assessing for nodepair in tqdm(nodepairs)]

100%|██████████| 1715286/1715286 [00:00<00:00, 3090986.12it/s]


In [14]:
joblib.dump(nodepairs, 'random/2/nodepairs.pkl', protocol=5)
joblib.dump(targets, 'random/2/targets.pkl', protocol=5)

['random/2/targets.pkl']

In [ ]:
%%time
nodepairs = np.array(nodepairs)
distances = np.array(distances)
targets = np.array(targets)
for select_distance in [3]:
  print(select_distance)
  filter_indices = (distances == select_distance)
  nodepairs[filter_indices].dump(f'{folder}{select_distance}/nodepairs.pkl')
  distances[filter_indices].dump(f'{folder}{select_distance}/distances.pkl')
  targets[filter_indices].dump(f'{folder}{select_distance}/targets.pkl')
  joblib.dump(g_learn, f'{folder}{select_distance}/graph.pkl')

In [15]:
print(f'{sum(targets) / len(nodepairs):e}')

3.527109e-04


## Distance analysis

In [ ]:
df = pd.DataFrame(dict(distances=distances, targets=targets))
df = df.groupby('distances')['targets'].agg(['size', 'sum'])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['size'], fill='tozeroy', name='# Nodepairs'))
fig.add_trace(go.Scatter(x=df.index, y=df['sum'], fill='tozeroy', name='# Positives'))
fig.update_layout(xaxis=dict(tickmode='linear', tick0=2, dtick=1), yaxis_type="log")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['size'].cumsum(), fill='tozeroy', name='# Nodepairs'))
fig.add_trace(go.Scatter(x=df.index, y=df['sum'].cumsum(), fill='tozeroy', name='# Positives'))
fig.update_layout(xaxis=dict(tickmode='linear', tick0=2, dtick=1), yaxis_type="log")

## Feature inspection

In [ ]:
%%time
df = joblib.load("random/2/features.pkl")

In [ ]:
fig = px.imshow(df.corr(), x=df.columns, y=df.columns)
fig.update_xaxes(side="top")

In [ ]:
def pairplot(df):
  return sns.pairplot(
    df[df['target']].append(df[~df['target']].sample(sum(df['target']))).apply(minmax_scale), 
    hue='target',
    hue_order=[True, False],
    palette={True: 'green', False: 'red'},
    kind='reg',
    diag_kws=dict(bw=.02),
    plot_kws=dict(scatter_kws=dict(alpha=.1))
  )

In [ ]:
pairplot(df)

## Hyperparameter selection

See `parameter_optimalization.ipynb`. We choose the following parameters:
- `max_depth = 1`
- `tree_method = 'hist'`
- `no feature scaling`

### XGBoost

$n=2$

In [152]:
def gridsearch(df: pd.DataFrame, random_state=1, also_random=True, max_depth=[1, 2]) -> pd.DataFrame:
  X = df.drop(columns='target').values
  y = df['target'].values
  
  param_grid=dict(max_depth=max_depth, scale_pos_weight=[sum(~y)/sum(y), 1])
  
  X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=1/3, random_state=random_state)
  clf = XGBClassifier(random_state=random_state, tree_method='hist', n_jobs=6)
  gridsearch = GridSearchCV(
    clf, 
    param_grid=param_grid, 
    scoring='average_precision', 
    n_jobs=30,
    cv=StratifiedKFold(shuffle=True, random_state=random_state),
    return_train_score=True
  )
  
  if also_random: 
    gridsearch_random = copy.deepcopy(gridsearch)
    np.random.seed(random_state)
    y_random = copy.deepcopy(y_trainval)
    np.random.shuffle(y_random)
  
  gridsearch.fit(X_trainval, y_trainval)
  df_dict = dict(
      mean_train=gridsearch.cv_results_['mean_train_score'],
      std_train=gridsearch.cv_results_['std_train_score'],
      mean_test=gridsearch.cv_results_['mean_test_score'],
      std_test=gridsearch.cv_results_['std_test_score'],
      test_fold0=gridsearch.cv_results_[f'split0_test_score'],
      test_fold1=gridsearch.cv_results_[f'split1_test_score'],
      test_fold2=gridsearch.cv_results_[f'split2_test_score'],
      test_fold3=gridsearch.cv_results_[f'split3_test_score'],
      test_fold4=gridsearch.cv_results_[f'split4_test_score']
  )
  
  if also_random: 
    gridsearch_random.fit(X_trainval, y_random)
    df_dict['mean_train_random']=gridsearch_random.cv_results_['mean_train_score']
    df_dict['std_train_random']=gridsearch_random.cv_results_['std_train_score']
    df_dict['mean_test_random']=gridsearch_random.cv_results_['mean_test_score']
    df_dict['std_test_random']=gridsearch_random.cv_results_['std_test_score']
  df = pd.DataFrame(df_dict, index=pd.Index([(d['max_depth'], d['scale_pos_weight'] > 1) for d in gridsearch.cv_results_['params']], name=('max_depth', 'balanced')))
  df['diff_train_test'] = (df['mean_test'] - df['mean_train']).abs()/df['mean_test']
  df['rstd_test'] = df['std_test'] / df['mean_test']
  if also_random: df['test_over_random'] = df['mean_test'] - df['mean_test_random']
  return df.sort_values('mean_test', ascending=False)
    
def report_performance(df: pd.DataFrame, random_state=1, max_depth=1, tree_method='hist', balanced=True, n_jobs=128):
  X = df.drop(columns='target').values
  y = df['target'].values
  X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=1/3, random_state=random_state)
  clf = XGBClassifier(max_depth=max_depth, n_jobs=128, tree_method=tree_method, scale_pos_weight=sum(~y)/sum(y) if balanced else 1 , random_state=random_state)
  clf.fit(X_trainval, y_trainval)
  y_pred = clf.predict_proba(X_test)[:,1]
  return average_precision_score(y_test, y_pred), roc_auc_score(y_test, y_pred)

In [ ]:
df = pd.read_pickle(f'temp/a1/2/features.pkl')
X = df.drop(columns='target').values
y = df['target'].values

param_grid=dict(max_depth=[1, 2], scale_pos_weight=[sum(~y)/sum(y), 1])

X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=1/3, random_state=1)
clf = XGBClassifier(random_state=1, tree_method='hist', n_jobs=6)
gridsearch = GridSearchCV(
  clf, 
  param_grid=param_grid, 
  scoring='average_precision', 
  n_jobs=30,
  cv=StratifiedKFold(shuffle=True, random_state=1),
  return_train_score=True
)
gridsearch.fit(X_trainval, y_trainval)

In [153]:
hps2 = gridsearch(pd.read_pickle(f'random/2/features.pkl'))

In [155]:
hps2[['mean_test', 'mean_train', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

mean_test  mean_train  diff_train_test  rstd_test  \
max_depth balanced                                                      
2         False      0.113700    0.131076         0.152823   0.039730   
          True       0.097232    0.104765         0.077468   0.042649   
1         False      0.069166    0.071592         0.035075   0.073890   
          True       0.062651    0.063839         0.018969   0.065853   

                    test_over_random  test_fold0  test_fold1  test_fold2  \
max_depth balanced                                                         
2         False             0.099973    0.110750    0.122274    0.109967   
          True              0.083471    0.093261    0.097869    0.103400   
1         False             0.055839    0.064274    0.076400    0.073122   
          True              0.049290    0.059332    0.065601    0.067685   

                    test_fold3  test_fold4  
max_depth balanced                          
2         False       0.114187    0.111323  
          True        0.092090    0.099541  
1         False       0.062946    0.069090  
          True        0.056500    0.064136

In [ ]:
hps3 = gridsearch(pd.read_pickle(f'temp/a1/3/features.pkl'))

In [ ]:
hps3[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

In [ ]:
hps4 = gridsearch(pd.read_pickle(f'temp/a1/4/features.pkl'))

In [ ]:
hps4[['mean_test', 'diff_train_test', 'rstd_test', 'test_over_random', 'test_fold0', 'test_fold1', 'test_fold2', 'test_fold3', 'test_fold4']]

In [157]:
report_performance(pd.read_pickle(f'random/2/features.pkl'), max_depth=1, balanced=False)

(0.06929365306164612, 0.7599842690910081)

In [ ]:
report_performance(pd.read_pickle(f'temp/a1/3/features.pkl'), balanced=True)

In [ ]:
report_performance(pd.read_pickle(f'temp/a1/4/features.pkl'), balanced=True)